# Begin by importing the necessary packages...

In [5]:
import pynwb
from neuroconv import NWBConverter, SpikeGLXRecordingInterface, MovieInterface

# Setup some file paths to point to example SpikeGLX/Neuropixels data, as well as some behavioral videos...

### The default path assumes you are running this notebook on the DANDIHub platform; adjust appropriately if running on a local device

In [16]:
spikeglx_file_path = "/shared/catalystneuro/ephy_testing_data/spikeglx/Noise4Sam_g0/Noise4Sam_g0_imec0/Noise4Sam_g0_t0.imec0.ap.bin"
video_file_path = "/shared/catalystneuro/behavior_testing_data/videos/CFR/video_mp4.mp4"

stub_path = "/home/jovyan/my_example_stubbed_spikeglx_and_behavioral_data.nwb"
nwbfile_path = "/home/jovyan/my_example_spikeglx_and_behavioral_data.nwb"

# To integrate multiple data streams, we use the NWBConverter

In [8]:
class MyConverter(NWBConverter):
    data_interface_classes = dict(
        Recording=SpikeGLXRecordingInterface,  # The names of the keys defined here will define how we reference the interfaces of this converter
        Movie=MovieInterface,
    )

In [12]:
source_data = dict(
    Recording=dict(file_path=spikeglx_file_path),  # The keys here must be the input keyword arguments to the individual interfaces
    Movie=dict(file_paths=[video_file_path]),  # Also, file-type inputs will always be named 'file_path'; folders will be 'folder_path'; ordered lists of multiple files will be 'file_path'
)

In [13]:
my_converter = MyConverter(source_data=source_data)

Source data is valid!


# All the previous commands for fetching metadata and running the conversion apply to the converter...

In [14]:
metadata = my_converter.get_metadata()

In [15]:
metadata

{'NWBFile': {'session_description': 'no description',
  'identifier': 'd65af50d-19de-4fd9-8084-de0ff02cfcd5',
  'session_start_time': '2020-11-03 10:35:10'},
 'Ecephys': {'Device': [{'name': 'Neuropixel-Imec',
    'description': '{"probe_type": "0", "probe_type_description": "NP1.0", "flex_part_number": "NP2_FLEX_0", "connected_base_station_part_number": "NP2_QBSC_00"}',
    'manufacturer': 'Imec'}],
  'ElectrodeGroup': [{'name': 's0',
    'description': 'a group representing shank s0',
    'location': 'unknown',
    'device': 'Neuropixel-Imec'}],
  'Electrodes': [{'name': 'shank_electrode_number',
    'description': '0-indexed channel within a shank.'},
   {'name': 'group_name',
    'description': 'Name of the ElectrodeGroup this electrode is a part of.'},
   {'name': 'contact_shapes', 'description': 'The shape of the electrode'}],
  'ElectricalSeries_raw': {'name': 'ElectricalSeries_raw',
   'description': 'Raw acquisition traces for the high-pass (ap) SpikeGLX data.'}},
 'Behavior':

# ... though interface-level conversion options like stub_test need to be specified...

In [26]:
nwbfile = my_converter.run_conversion(
    nwbfile_path=stub_path,
    metadata=metadata,
    conversion_options=dict(
        Recording=dict(stub_test=True),
        Movie=dict(stub_test=True),
    )
)

Metadata is valid!
conversion_options is valid!


retrieving timestamps: 100%|██████████| 152/152 [00:00<00:00, 1660.61it/s]


NWB file saved at /home/jovyan/my_example_stubbed_spikeglx_and_behavioral_data.nwb!


# ...the in-memory copy of the NWBFile is returned for easy access...

In [29]:
print(nwbfile)
print(nwbfile.acquisition["Video: video_mp4"])

root pynwb.file.NWBFile at 0x139963973289488
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>,
    Video: video_mp4 <class 'pynwb.image.ImageSeries'>
  }
  devices: {
    Neuropixel-Imec <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    s0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2022, 7, 21, 18, 48, 38, 93481, tzinfo=tzlocal())]
  identifier: d65af50d-19de-4fd9-8084-de0ff02cfcd5
  session_description: no description
  session_start_time: 2020-11-03 10:35:10+00:00
  timestamps_reference_time: 2020-11-03 10:35:10+00:00

Video: video_mp4 pynwb.image.ImageSeries at 0x139963972735376
Fields:
  comments: no comments
  conversion: 1.0
  description: Video recorded by camera.
  external_file: ['/shared/catalystneuro/behavior_testing_data/videos/CFR/video_mp4.mp4']
  format: external
  offset: 0.0
  rate: 149.99250037498123
  resolution:

# ... now we can run the entire conversion!

In [33]:
_ = my_converter.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata)

Metadata is valid!
conversion_options is valid!


retrieving timestamps: 100%|██████████| 152/152 [00:00<00:00, 1637.92it/s]


NWB file saved at /home/jovyan/my_example_spikeglx_and_behavioral_data.nwb!


# Bonus: external files and movies

### This will come up in more detail tomorrow during the DANDI usage tutorial, but it's worth pointing out here...
### Movie contents can be written directly to NWBFiles, but compression is often better in the original format...
### Thus we give users the choice of pointing to an 'external_file'...
### However, there is one very important Best Practice to follow when sharing (or uploading) NWBFiles containing external links - the file path to the video should be written as a relative path (not an absolute one) and the NWBFile should generally be written/stored adjacent to the external reference

### The MovieInterface from NeuroConv supports sophisticated writing tools for copying the data from a common video format (.avi, .mp4, etc.), but uses 'external_file' by default.